# Urdu & Pashto Tokenization Notebook

This notebook demonstrates whitespace vs regex-based tokenization for Urdu text using the column **News Text**.

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import re
import sentencepiece as spm
import os
from transformers import AutoTokenizer

In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!pip install ./indic_nlp_library


fatal: destination path 'indic_nlp_library' already exists and is not an empty directory.
Processing ./indic_nlp_library
  Preparing metadata (setup.py) ... done
  Created wheel for indic_nlp_library: filename=indic_nlp_library-0.92-py3-none-any.whl size=40778 sha256=24074fd3a31e12134d0d160d1cbcb81200f9c77e8ad3ff714acd8547a9644b8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-p99bpkj_/wheels/9a/20/c2/909246e3fb53bf4c760f1ada72e462b85e3588ed01d1ae2169
Successfully built indic_nlp_library
  Attempting uninstall: indic_nlp_library
    Found existing installation: indic_nlp_library 0.92
    Uninstalling indic_nlp_library-0.92:
      Successfully uninstalled indic_nlp_library-0.92


In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git /content/indic_nlp_resources

fatal: destination path '/content/indic_nlp_resources' already exists and is not an empty directory.


In [ ]:
from indicnlp.tokenize import indic_tokenize

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import pandas as pd

file_path = '/content/drive/MyDrive/Colab Notebooks/dataset/UrduDataset.xlsx'

try:
    df = pd.read_excel(file_path)
    print("Dataset loaded successfully:")

except FileNotFoundError:
    print(f"Error: The file at '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Dataset loaded successfully:


In [ ]:

print(df.columns)


Index(['Index', 'Headline', 'News Text', 'Category', 'Date', 'URL', 'Source',
       'News length'],
      dtype='object')


## Whitespace Tokenization (Not Recommended)

In [ ]:

df['whitespace_tokens'] = df['News Text'].astype(str).apply(lambda x: x.split())
df[['News Text', 'whitespace_tokens']].head()


,News Text,whitespace_tokens
0,اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...,"[اسلام, باد, عالمی, بینک, خیبرپختونخوا, کے, قب..."
1,اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...,"[اسلام, باد, فیڈرل, بورڈ, ریونیو, ایف, بی, نے,..."
2,اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...,"[اسلام, باد, بورڈ, انویسٹمنٹ, بی, او, ئی, کے, ..."
3,اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...,"[اسلام, اباد, پاکستان, میں, ماہ, نومبر, میں, م..."
4,اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...,"[اسلام, باد, نیشنل, ٹرانسمیشن, اینڈ, ڈسپیچ, کم..."


## Regex-based Unicode Tokenization (Recommended)

In [ ]:

import re

def regex_tokenize(text):
    pattern = r'[\u0600-\u06FF]+'
    return re.findall(pattern, str(text))

df['regex_tokens'] = df['News Text'].apply(regex_tokenize)
df[['News Text', 'regex_tokens']].head()


# **Indic NLP tokenization**

In [ ]:
df['indic_tokens'] = df['News Text'].astype(str).apply(lambda x: indic_tokenize.trivial_tokenize(x))
df['indic_tokens']

# Tokenize Urdu using BERT tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
df['bert_encoding'] = df['News Text'].astype(str).apply(
    lambda x: tokenizer(
        x,
        truncation=True,
        max_length=512,
        padding=False,
        return_attention_mask=False,
        return_token_type_ids=False
    )
)

# Extract tokens safely
df['bert_tokens'] = df['bert_encoding'].apply(lambda x: tokenizer.convert_ids_to_tokens(x['input_ids']))
df['bert_len'] = df['bert_tokens'].apply(len)



**Calculate The Tokens**

In [ ]:
# 3. Calculate token lengths
# -------------------------------
df['whitespace_len'] = df['whitespace_tokens'].apply(len)
df['bert_len'] = df['bert_tokens'].apply(len)
df['indic_len'] = df['indic_tokens'].apply(len)
df['regex_len'] = df['regex_tokens'].apply(len)

## Token Length Comparison

In [ ]:
df[['News Text', 'whitespace_len', 'regex_len', 'indic_len','bert_len']].head()

,News Text,whitespace_len,regex_len,indic_len,bert_len
0,اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...,375,370,375,512
1,اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...,429,412,429,512
2,اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...,446,445,446,512
3,اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...,490,446,490,512
4,اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...,588,576,588,512


In [ ]:
# 4. Preview tokenized dataset
# -------------------------------
df[['News Text', 'whitespace_tokens', 'regex_tokens', 'indic_tokens', 'bert_tokens']].head()

,News Text,whitespace_tokens,regex_tokens,indic_tokens,bert_tokens
0,اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...,"[اسلام, باد, عالمی, بینک, خیبرپختونخوا, کے, قب...","[اسلام, باد, عالمی, بینک, خیبرپختونخوا, کے, قب...","[اسلام, باد, عالمی, بینک, خیبرپختونخوا, کے, قب...","[[CLS], اسلام, با, ##د, عالمی, بین, ##ک, خ, ##..."
1,اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...,"[اسلام, باد, فیڈرل, بورڈ, ریونیو, ایف, بی, نے,...","[اسلام, باد, فیڈرل, بورڈ, ریونیو, ایف, بی, نے,...","[اسلام, باد, فیڈرل, بورڈ, ریونیو, ایف, بی, نے,...","[[CLS], اسلام, با, ##د, فی, ##ڈر, ##ل, بو, ##ر..."
2,اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...,"[اسلام, باد, بورڈ, انویسٹمنٹ, بی, او, ئی, کے, ...","[اسلام, باد, بورڈ, انویسٹمنٹ, بی, او, ئی, کے, ...","[اسلام, باد, بورڈ, انویسٹمنٹ, بی, او, ئی, کے, ...","[[CLS], اسلام, با, ##د, بو, ##ر, ##ڈ, ان, ##وی..."
3,اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...,"[اسلام, اباد, پاکستان, میں, ماہ, نومبر, میں, م...","[اسلام, اباد, پاکستان, میں, ماہ, نومبر, میں, م...","[اسلام, اباد, پاکستان, میں, ماہ, نومبر, میں, م...","[[CLS], اسلام, اب, ##اد, پاکستان, میں, ما, ##ہ..."
4,اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...,"[اسلام, باد, نیشنل, ٹرانسمیشن, اینڈ, ڈسپیچ, کم...","[اسلام, باد, نیشنل, ٹرانسمیشن, اینڈ, ڈسپیچ, کم...","[اسلام, باد, نیشنل, ٹرانسمیشن, اینڈ, ڈسپیچ, کم...","[[CLS], اسلام, با, ##د, ن, ##یش, ##نل, ٹ, ##را..."


## Save Tokenized Dataset

In [ ]:

output_path = '/content/drive/MyDrive/Colab Notebooks/dataset/UrduDataset_Tokenized.xlsx'
df.to_excel(output_path, index=False)
print("Tokenized dataset saved successfully!")


Tokenized dataset saved successfully!
